<a href="https://colab.research.google.com/github/Parth-Ach2002/Image-Classification/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing the necessary libraries**

In [4]:
import tensorflow as tf
import keras
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

**Connecting google drive with our file in order to access the contents of the dataset**

In [1]:
# Mounting the Data from Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Setting the path for all the datasets**

In [2]:
# Set path for all three datasets
train_path="/content/drive/MyDrive/Image Classification Dataset/Train"
test_path="/content/drive/MyDrive/Image Classification Dataset/Test"
val_path="/content/drive/MyDrive/Image Classification Dataset/Validation"

**Generating image data along with few transformations**

In [6]:
train_datagen = ImageDataGenerator(rescale = 1./255, rotation_range=30, width_shift_range=10.0, height_shift_range=10.0, shear_range=10,
                                   zoom_range=0.1,
                                   vertical_flip=True)
test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)
